## Load the dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('FakeNews_Data/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [5]:
df.shape

(20800, 5)

In [6]:
# copy df to data
data = df.copy()

In [7]:
data.shape

(20800, 5)

### drop the null values

In [8]:
data = data.dropna()

In [9]:
data.shape

(18285, 5)

### get the independent and dependent features

In [10]:
X = data.drop('label', axis=1)
y = data['label']

In [11]:
print(X.shape)
X.head()

(18285, 4)


,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
print(y.shape)
y.head()

(18285,)


0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [13]:
### initialize vocab size
vocab_size = 10000

## Onehot Representation

In [14]:
messages = X.copy()

In [15]:
messages.columns

Index(['id', 'title', 'author', 'text'], dtype='object')

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

### Dataset Preprocessing using stemming

In [18]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
len(corpus)

18285

In [21]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

### Onehot representation of corpus

In [23]:
onehot_repre = [one_hot(words, vocab_size) for words in corpus]
onehot_repre

[[5577, 2112, 950, 8436, 8614, 5885, 7401, 1322, 2609, 9635],
 [4251, 3672, 1702, 8145, 1698, 8188, 8268],
 [8863, 3919, 6163, 8659],
 [9645, 8154, 7803, 9675, 3317, 1161],
 [3783, 1698, 7675, 7146, 8668, 2094, 1698, 1740, 1466, 2131],
 [4715,
  2903,
  9920,
  5579,
  8882,
  8252,
  8627,
  6562,
  4993,
  7330,
  1842,
  5903,
  3050,
  8006,
  8268],
 [3360, 5132, 5958, 5860, 5862, 6196, 2934, 1517, 2685, 6031, 6220],
 [7744, 1602, 7793, 114, 3571, 333, 8252, 3796, 2685, 6031, 6220],
 [1982, 1223, 6947, 28, 8248, 5559, 5011, 4017, 8252, 7779],
 [5908, 8291, 2295, 4102, 8436, 5407, 3570, 9414],
 [4775, 6651, 1088, 5595, 6442, 5142, 680, 9780, 7583, 5127, 3383],
 [9675, 7462, 8614, 5559, 8252, 3571],
 [4588, 1468, 7782, 4371, 9970, 1656, 2028, 2436, 2880],
 [937, 1562, 5044, 7970, 7351, 6031, 2174, 2685, 6031, 6220],
 [2163, 266, 9819, 810, 2112, 2685, 6031, 6220],
 [839, 8984, 6803, 3154, 3516, 7482, 231, 5865, 1281, 4766],
 [6292, 8605, 3672],
 [3623, 847, 9295, 9228, 8252, 5311, 5

### Embedding Representation of onehot encoding

In [24]:
sentence_len = 20
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sentence_len)
print(embedded_docs)

[[   0    0    0 ... 1322 2609 9635]
 [   0    0    0 ... 1698 8188 8268]
 [   0    0    0 ... 3919 6163 8659]
 ...
 [   0    0    0 ... 2685 6031 6220]
 [   0    0    0 ... 6453 8716 9507]
 [   0    0    0 ... 4339 7800 6088]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 5577,
       2112,  950, 8436, 8614, 5885, 7401, 1322, 2609, 9635])

## Created the Sequential Model

In [26]:
# creating LSTM model 
dim = 40
model = Sequential()
model.add(Embedding(vocab_size, dim, input_length=sentence_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
## Creating Bidirectional model
dim = 40
model1=Sequential()
model1.add(Embedding(vocab_size,dim,input_length=sentence_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
print(len(embedded_docs))
print(y.shape)

18285
(18285,)


In [29]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [30]:
print(X_final.shape)
print(y_final.shape)

(18285, 20)
(18285,)


## Bidirection LSTM Model Training

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, train_size=0.8, random_state=42)

In [32]:
# train model
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 17s 74ms/step - loss: 0.2881 - accuracy: 0.8670 - val_loss: 0.2009 - val_accuracy: 0.9136
Epoch 2/10
229/229 [==============================] - 16s 68ms/step - loss: 0.1215 - accuracy: 0.9532 - val_loss: 0.2280 - val_accuracy: 0.9128
Epoch 3/10
229/229 [==============================] - 17s 75ms/step - loss: 0.0778 - accuracy: 0.9714 - val_loss: 0.2404 - val_accuracy: 0.9122
Epoch 4/10
229/229 [==============================] - 17s 76ms/step - loss: 0.0504 - accuracy: 0.9824 - val_loss: 0.3571 - val_accuracy: 0.9089
Epoch 5/10
229/229 [==============================] - 17s 74ms/step - loss: 0.0331 - accuracy: 0.9888 - val_loss: 0.3936 - val_accuracy: 0.9070
Epoch 6/10
229/229 [==============================] - 17s 76ms/step - loss: 0.0256 - accuracy: 0.9913 - val_loss: 0.4047 - val_accuracy: 0.9098
Epoch 7/10
229/229 [==============================] - 16s 70ms/step - loss: 0.0213 - accuracy: 0.9923 - val_loss: 0.4408 - val_accuracy:

### prediction

In [41]:
y_pred1 = model1.predict_classes(X_test)

### Performance Metrics and Accuracy

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [44]:
cm = confusion_matrix(y_test, y_pred1)
cm

array([[1909,  173],
       [ 161, 1414]], dtype=int64)

In [45]:
accuracy = accuracy_score(y_test, y_pred1)
accuracy

0.9086683073557561

In [46]:
report = classification_report(y_test, y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      2082
           1       0.89      0.90      0.89      1575

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

